In [1]:
#day5
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
import torch.optim as optim

 
# 数据下载和标准化
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=4, shuffle=True, num_workers=0)
test_set = torchvision.datasets.CIFAR10(root='./data', train=False, download=False, transform=transform)
testloader = DataLoader(test_set, batch_size=4, shuffle=False, num_workers=0)
 
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
 
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
 

class CNNNet_1(nn.Module):
    """
    构建卷积神经网络
    搭建 三个conv1 conv2 conv3 层 和两个pool 层 两个全连接层
    """
    def __init__(self):
        super(CNNNet_1, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=5, stride=1)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=36, kernel_size=3, stride=1)
        self.conv3 = nn.Conv2d(in_channels=36, out_channels=64, kernel_size=1, stride=1)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 6 * 6, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = F.relu(self.conv3(x))
        x = x.view(-1, 64 * 6 * 6)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
# 小卷积核实验
net_small_kernel = CNNNet_1()
optimizer = optim.SGD(net_small_kernel.parameters(), lr=0.001, momentum=0.9)
criterion = nn.CrossEntropyLoss()


for epoch in range(10):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # 获取训练数据
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # 权重参数梯度清零
        optimizer.zero_grad()

        # 正向和反向传播
        outputs = net_small_kernel(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # 显示损失值
        running_loss += loss.item()
        if i % 2000 == 1999:
            print('[%d,%5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

correct = 0
tatal = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = net_small_kernel(images)
        _, predicted = torch.max(outputs.data, 1)
        tatal += labels.size(0)
        correct += (predicted == labels).sum().item()
print('Accuracy of the netwaork on the 10000 test images: %d %%' % (100 * correct / tatal))

# 大卷积核实验
net_large_kernel = CNNNet_1()
net_large_kernel.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=11, stride=1, padding=5)
net_large_kernel.conv2 = nn.Conv2d(in_channels=16, out_channels=36, kernel_size=9, stride=1, padding=4)
net_large_kernel.conv3 = nn.Conv2d(in_channels=36, out_channels=64, kernel_size=7, stride=1, padding=3)
optimizer = optim.SGD(net_large_kernel.parameters(), lr=0.001, momentum=0.9)

for epoch in range(10):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # 获取训练数据
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # 权重参数梯度清零
        optimizer.zero_grad()

        # 正向和反向传播
        outputs = net_large_kernel(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # 显示损失值
        running_loss += loss.item()
        if i % 2000 == 1999:
            print('[%d,%5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

correct = 0
tatal = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = net_large_kernel(images)
        _, predicted = torch.max(outputs.data, 1)
        tatal += labels.size(0)
        correct += (predicted == labels).sum().item()
print('Accuracy of the netwaork on the 10000 test images: %d %%' % (100 * correct / tatal))

Files already downloaded and verified
[1, 2000] loss: 2.165
[1, 4000] loss: 1.711
[1, 6000] loss: 1.527
[1, 8000] loss: 1.475
[1,10000] loss: 1.384
[1,12000] loss: 1.334
[2, 2000] loss: 1.235
[2, 4000] loss: 1.189
[2, 6000] loss: 1.151
[2, 8000] loss: 1.126
[2,10000] loss: 1.081
[2,12000] loss: 1.086
[3, 2000] loss: 0.970
[3, 4000] loss: 0.964
[3, 6000] loss: 0.971
[3, 8000] loss: 0.938
[3,10000] loss: 0.947
[3,12000] loss: 0.925
[4, 2000] loss: 0.814
[4, 4000] loss: 0.815
[4, 6000] loss: 0.819
[4, 8000] loss: 0.831
[4,10000] loss: 0.800
[4,12000] loss: 0.846
[5, 2000] loss: 0.690
[5, 4000] loss: 0.698
[5, 6000] loss: 0.734
[5, 8000] loss: 0.738
[5,10000] loss: 0.713
[5,12000] loss: 0.737
[6, 2000] loss: 0.593
[6, 4000] loss: 0.600
[6, 6000] loss: 0.614
[6, 8000] loss: 0.647
[6,10000] loss: 0.642
[6,12000] loss: 0.667
[7, 2000] loss: 0.459
[7, 4000] loss: 0.495
[7, 6000] loss: 0.542
[7, 8000] loss: 0.578
[7,10000] loss: 0.583
[7,12000] loss: 0.592
[8, 2000] loss: 0.399
[8, 4000] loss: 

RuntimeError: shape '[-1, 2304]' is invalid for input of size 16384